In [1]:
import logging
import os
import sys
import time

import torch
import torchaudio
from datasets import load_dataset
from IPython.display import Audio
from huggingface_hub import hf_hub_download

os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "0"
os.environ["SAM_AUDIO_LOAD_VERBOSE"] = "1"

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler(sys.stdout)],
    force=True,
)
logging.getLogger().setLevel(logging.INFO)
logging.getLogger("sam_audio").setLevel(logging.INFO)
logging.getLogger("sam_audio.model.base").setLevel(logging.INFO)
logging.getLogger("sam_audio.processor").setLevel(logging.INFO)
logging.getLogger("huggingface_hub").setLevel(logging.INFO)
logging.getLogger("transformers").setLevel(logging.WARNING)

from sam_audio import SAMAudio, SAMAudioProcessor

import sam_audio as _sam_audio
print(f"[span_prompting] sam_audio import path: {_sam_audio.__file__}", flush=True)


/home/admin-grant-jr/github/sam-audio/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/home/admin-grant-jr/github/sam-audio/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[span_prompting] sam_audio import path: /home/admin-grant-jr/github/sam-audio/sam_audio/__init__.py


/home/admin-grant-jr/github/sam-audio/.venv/lib/python3.12/site-packages/imagebind/data.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
# Create a synthetic 8-second audio clip with a horn-like tone around the anchor window
sample_rate = 48_000
duration = 8.0
num_samples = int(sample_rate * duration)
# Base noise floor
wav = 0.01 * torch.randn(num_samples)
# Add a horn-like sine burst between 6.3s and 7.0s
start = int(6.3 * sample_rate)
end = int(7.0 * sample_rate)
t = torch.linspace(0, (end - start) / sample_rate, end - start, dtype=torch.float32)
wav[start:end] += 0.2 * torch.sin(2 * torch.pi * 220 * t)
# Shape to (channels, time)
samples = wav.unsqueeze(0)
Audio(wav.numpy(), rate=sample_rate)


In [3]:
import threading

from huggingface_hub import snapshot_download

model_id = "facebook/sam-audio-large"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logging.info("Using device: %s", device)

print(f"[span_prompting] Ensuring snapshot is available locally for {model_id}...", flush=True)
t0 = time.time()
model_dir = snapshot_download(repo_id=model_id)
print(f"[span_prompting] Snapshot ready at {model_dir} ({time.time() - t0:.1f}s)", flush=True)

print("[span_prompting] Loading model checkpoint (this can be slow)...", flush=True)
_stop = threading.Event()

def _heartbeat(label: str):
    while not _stop.wait(10):
        print(f"[span_prompting] {label} (still working...)", flush=True)

_thread = threading.Thread(target=_heartbeat, args=("Loading model",), daemon=True)
_thread.start()
try:
    t0 = time.time()
    model = SAMAudio.from_pretrained(
        model_dir,
        visual_ranker=None,
        text_ranker=None,
        span_predictor=None,
    ).to(device).eval()
finally:
    _stop.set()
    _thread.join(timeout=0.1)

print(f"[span_prompting] Model loaded ({time.time() - t0:.1f}s)", flush=True)

print("[span_prompting] Loading processor config...", flush=True)
t0 = time.time()
processor = SAMAudioProcessor.from_pretrained(model_dir)
print(f"[span_prompting] Processor loaded ({time.time() - t0:.1f}s)", flush=True)


2025-12-19 23:19:44,181 | INFO | root | Using device: cuda
[span_prompting] Ensuring snapshot is available locally for facebook/sam-audio-large...


Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 32615.12it/s]

[span_prompting] Snapshot ready at /home/admin-grant-jr/.cache/huggingface/hub/models--facebook--sam-audio-large/snapshots/af8ba1712fceaf0850b06b06d046bcf6ac24c840 (0.2s)
[span_prompting] Loading model checkpoint (this can be slow)...
2025-12-19 23:19:44,435 | INFO | sam_audio.model.base | Loading /home/admin-grant-jr/.cache/huggingface/hub/models--facebook--sam-audio-large/snapshots/af8ba1712fceaf0850b06b06d046bcf6ac24c840 (cache_dir=None, local_files_only=False, force_download=False, revision=None)
[sam_audio.model.base] Loading /home/admin-grant-jr/.cache/huggingface/hub/models--facebook--sam-audio-large/snapshots/af8ba1712fceaf0850b06b06d046bcf6ac24c840 (cache_dir=None, local_files_only=False, force_download=False, revision=None)
2025-12-19 23:19:44,437 | INFO | sam_audio.model.base | Reading config.json from /home/admin-grant-jr/.cache/huggingface/hub/models--facebook--sam-audio-large/snapshots/af8ba1712fceaf0850b06b06d046bcf6ac24c840
[sam_audio.model.base] Reading config.json fro

[span_prompting] Loading model (still working...)
[span_prompting] Loading model (still working...)
[span_prompting] Loading model (still working...)
[span_prompting] Loading model (still working...)
2025-12-19 23:20:28,606 | INFO | sam_audio.model.base | Loading checkpoint from /home/admin-grant-jr/.cache/huggingface/hub/models--facebook--sam-audio-large/snapshots/af8ba1712fceaf0850b06b06d046bcf6ac24c840/checkpoint.pt (14172.9 MB)
[sam_audio.model.base] Loading checkpoint from /home/admin-grant-jr/.cache/huggingface/hub/models--facebook--sam-audio-large/snapshots/af8ba1712fceaf0850b06b06d046bcf6ac24c840/checkpoint.pt (14172.9 MB)
[span_prompting] Loading model (still working...)
[span_prompting] Loading model (still working...)
[span_prompting] Loading model (still working...)
[span_prompting] Loading model (still working...)


IOStream.flush timed out


[span_prompting] Loading model (still working...)
[span_prompting] Loading model (still working...)
[span_prompting] Loading model (still working...)
[span_prompting] Loading model (still working...)
[span_prompting] Loading model (still working...)
[span_prompting] Loading model (still working...)
2025-12-19 23:22:45,336 | INFO | sam_audio.model.base | Checkpoint loaded (139.3s)
[sam_audio.model.base] Checkpoint loaded (139.3s)
2025-12-19 23:22:45,367 | INFO | sam_audio.model.base | Applying state dict (strict=True)
[sam_audio.model.base] Applying state dict (strict=True)
[span_prompting] Loading model (still working...)
[span_prompting] Loading model (still working...)
[span_prompting] Loading model (still working...)
[span_prompting] Loading model (still working...)
[span_prompting] Loading model (still working...)
[span_prompting] Loading model (still working...)
[span_prompting] Loading model (still working...)
[span_prompting] Loading model (still working...)
[span_prompting] Loa

IOStream.flush timed out


[span_prompting] Loading model (still working...)
[span_prompting] Loading model (still working...)
2025-12-19 23:27:19,374 | INFO | sam_audio.model.base | State dict applied (278.4s)
[sam_audio.model.base] State dict applied (278.4s)
[span_prompting] Loading model (still working...)
[span_prompting] Loading model (still working...)
[span_prompting] Loading model (still working...)
[span_prompting] Loading model (still working...)
[span_prompting] Loading model (still working...)
[span_prompting] Loading model (still working...)
[span_prompting] Loading model (still working...)
[span_prompting] Loading model (still working...)
[span_prompting] Loading model (still working...)
[span_prompting] Loading model (still working...)
[span_prompting] Loading model (still working...)
[span_prompting] Loading model (still working...)
[span_prompting] Model loaded (581.1s)
[span_prompting] Loading processor config...
2025-12-19 23:29:25,587 | INFO | sam_audio.processor | Reading config.json from /

In [4]:
wav = torchaudio.functional.resample(
    samples, sample_rate, processor.audio_sampling_rate
)
wav = wav.mean(0, keepdim=True)
inputs = processor(
    audios=[wav], descriptions=["A horn honking"], anchors=[[ ["+", 6.3, 7.0] ]]
).to(device)

t0 = time.time()
with torch.inference_mode():
    result = model.separate(
        inputs,
        ode_opt={"method": "midpoint", "options": {"step_size": 2 / 8}},
    )
print(f"Separation finished in {time.time() - t0:.1f}s")


Separation finished in 192.6s


In [5]:
Audio(result.target[0].cpu(), rate=processor.audio_sampling_rate)